#### 1.0 运行环境

In [ ]:
import sys
sys.path.append("../")
from common.env import AppConfig
from common.env import LocalLLM
from llama_index.core import Settings
config = AppConfig(local_llm=LocalLLM.LM_STUDIO)
logger = config.logger

#### 2. chat with database by an agent

In [ ]:
from llama_index.core.agent import AgentRunner
from llama_index.agent.openai import OpenAIAgentWorker
from llama_index.tools.database import DatabaseToolSpec
from llama_index.core import Settings
from IPython.display import Markdown, display
db_tools = DatabaseToolSpec(uri="sqlite:///database//employees.db")
tools = db_tools.to_tool_list()

step_engine = OpenAIAgentWorker.from_tools(llm= Settings.llm, 
                                           tools=tools, 
                                           verbose=True)
agent = AgentRunner(step_engine)
input =  (
    "Find the highest paid HR employee and write "
    "them an email announcing a bonus"
)

task = agent.create_task(input)
step_output = agent.run_step(task.task_id)

while not step_output.is_last:
    step_output = agent.run_step(task.task_id)

response = agent.finalize_response(task.task_id)
display(Markdown(f'{response}'))